# How to use few shot prompting

In [10]:
from dotenv import load_dotenv
from os import getenv

load_dotenv()

if getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI api key is required")

### Example 1

- For complex tool use it's is very useful to add [few-shot example](./few_shot_prompting.md/#few-shot-prompting) to the prompt. We can do this by adding `AIMessage` and `ToolCalls` and corresponding `ToolMessage` to our prompt

In [14]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """add two numbers"""

    return a * b


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""

    return a * b


tools = [multiply, add]
tools_by_name = {t.name: t for t in tools}

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)

In [12]:
query = "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_4H6287LSjPOjdcUaoTHXdDAn',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': -20, 'b': 0},
  'id': 'call_Q9cXj3y7DaAeL11Rrmb7JP46',
  'type': 'tool_call'}]

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

example = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "Multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "Add", "args": {
            "x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 
Use past tool usage as an example of how to correctly use the tools."""

few_shot_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=system),
    *example,
    ("human", "{query}")
])

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
ai_msg = chain.invoke("what is 119 times 8 minus 20")

content='' additional_kwargs={'tool_calls': [{'id': 'call_znWENPEntrC6M1J2jlZcR0Ug', 'function': {'arguments': '{"a": 119, "b": 8}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_Ns8PNI7GB6S4OBAKN7Znt8Yp', 'function': {'arguments': '{"a": -20, "b": 0}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 221, 'total_tokens': 273, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-2f0d1113-fad0-4aba-b1e9-c0a3383005c3-0' tool_calls=[{'name': 'multiply', 'args': {'a': 119, 'b': 8}, 'id': 'call_znWENPEntrC6M1J2jlZcR0Ug', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': -20, 'b': 0}, 'id': 'call_Ns8PNI7GB6S4OBAKN7Z

In [27]:
for t in ai_msg.tool_calls:
    selected_tool = tools_by_name[t["name"]]
    tool_result = selected_tool.invoke(t)

    few_shot_prompt.invoke(tool_result)

In [28]:
chain.invoke(few_shot_prompt)

AIMessage(content='To find the product of 317253 and 128472 plus four, I used the calculator tools as follows:\n\n1. **Multiplication**: I first multiplied 317253 by 128472 using the `Multiply` tool. The result was 16505054784.\n   \n2. **Addition**: Then, I added 4 to the result of the multiplication using the `Add` tool. The final result was 16505054788.\n\nSo, the complete calculation gives us that the product of 317253 and 128472 plus four is 16505054788.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 502, 'total_tokens': 623, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None}, id='run-cdca0988-aa87-46a4-b65a-456805802461-0', usage_metadata=